In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Importing the required packages 
import numpy as np 
import pandas as pd 

In [ ]:
from sklearn.metrics import confusion_matrix 
from sklearn.model_selection import train_test_split 
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score 

In [ ]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

# Impute missing values using Iterative Imputer

In [ ]:
GDM_final = pd.read_csv('../input/gdm-data/gdm_final.csv')
GDM_final

In [ ]:
# Finding what all columns have null values
GDM_final.columns[GDM_final.isnull().any()]

In [ ]:
# Target column should not have null values so removing from features list
X = GDM_final.drop('Glucose Result', axis =1)

Y = GDM_final['Glucose Result']

In [ ]:
corr = X.corr()

In [ ]:
import seaborn as sns
from matplotlib import pyplot as plt
plt.figure(figsize= (20,22))
sns.set(font_scale=1.5)
sns.heatmap(corr, annot=True)
plt.show()

In [ ]:
# define imputer
imputer = IterativeImputer() #Bayesian ridge regression by default
# fit on the dataset
imputer.fit(X)
# transform the dataset
Xtrans = imputer.transform(X)
Xtrans = pd.DataFrame(Xtrans)
Xtrans.columns = X.columns

In [ ]:
# Finding what all columns have null values
Xtrans.columns[Xtrans.isnull().any()]
Xtrans.to_csv("nonull.csv")

# Predicitng GDM using the dataset with imputed values

In [ ]:
GDM_Nonull = pd.read_csv('../input/gdm-nonull/GDM_nonull.csv')
GDM_Nonull

In [ ]:
X = GDM_Nonull.drop('Glucose Result', axis =1)

Y = GDM_Nonull['Glucose Result']

In [ ]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.25)

In [ ]:
RF = RandomForestClassifier(n_estimators=5,random_state=0)

In [ ]:
RF.fit(X_train, Y_train) 

In [ ]:
pred = RF.predict(X_test)
pred

In [ ]:
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(Y_test, pred))

In [ ]:
RF.estimators_[0]

In [ ]:
from sklearn import tree
import graphviz

# plt.figure(figsize= (22,20))
# tree.plot_tree(RF.estimators_[0],filled=True)

#Print tree
feature_names = ['Pulse','BMI','Smoking','Ethnicity','PreviousGDM','Age >30','High Risk','HBa1C',
                     'Creatinine','U Protein', 'U Creatinine']
target_names = ['GDM-Yes','GDM-No']
    
dot_data = tree.export_graphviz(RF.estimators_[0], out_file=None,
                                    feature_names=feature_names,
                                    class_names=target_names, filled = True)
#Draw tree
tr = graphviz.Source(dot_data, format ="png")
tr

#Build all the trees
for i in range(len(RF.estimators_)):
    print(tree.export_text(RF.estimators_[i]))
  

In [ ]:
importance = RF.feature_importances_

In [ ]:
feature_importance = pd.Series(importance, index=X.columns)
feature_importance.nlargest(10).plot(kind='barh')

In [ ]:
print(metrics.classification_report(Y_test, pred))
precision, recall, _ = metrics.precision_recall_curve(Y_test, pred, pos_label=1)
average_precision = metrics.average_precision_score(Y_test, pred)
print (precision)